In [58]:
import multiprocessing as mp
import numpy as np
import pod5 as p5
import pysam
from multiprocessing import Queue
import time
import os
import signal
from tqdm import tqdm
def handler(signum, frame):
    print("hello world!")
signal.signal(signal.SIGPIPE, signal.SIG_IGN)  # 忽略SIGPIPE信号

<Handlers.SIG_IGN: 1>

In [5]:
def extract_signal_from_pod5_dict(pod5_path):
    signals={}
    with p5.Reader(pod5_path) as reader:
        for read_record in reader.reads():
            signals[str(read_record.read_id)] = {'signal':read_record.signal,'shift':read_record.calibration.offset,'scale':read_record.calibration.scale}#不加str会变成UUID，很奇怪
    return signals

def extract_move_from_bam_dict(bam_path):
    read_dict=dict()
    bamfile = pysam.AlignmentFile(bam_path, "rb",check_sq=False)
    try:
        for read in bamfile.fetch(until_eof=True):#暂时不使用索引，使用返回是空值
            print(read.query_name)
            tags=dict(read.tags)
            mv_tag=tags['mv']
            ts_tag=tags['ts']
            sm_tag=tags["sm"]
            sd_tag=tags["sd"]
            read_dict.update({read.query_name:{"sequence":read.query_sequence,"stride":mv_tag[0],"mv_table":np.array(mv_tag[1:]),"num_trimmed":ts_tag,"shift":sm_tag,"scale":sd_tag}})
    except ValueError:
        print('bam don\'t has index')
        for read in bamfile.fetch(until_eof=True,multiple_iterators=False):
            tags=dict(read.tags)
            mv_tag=tags['mv']
            ts_tag=tags['ts']
            sm_tag=tags["sm"]
            sd_tag=tags["sd"]
            read_dict[read.query_name] = {"sequence":read.query_sequence,"stride":mv_tag[0],"mv_table":np.array(mv_tag[1:]),"num_trimmed":ts_tag,"shift":sm_tag,"scale":sd_tag}
    return read_dict


In [15]:
def extract_signal_from_pod5(pod5_path):
    signals=[]
    with p5.Reader(pod5_path) as reader:
        for read_record in reader.reads():
            #signals[str(read_record.read_id)] = {'signal':read_record.signal,'shift':read_record.calibration.offset,'scale':read_record.calibration.scale}#不加str会变成UUID，很奇怪
            signals.append([str(read_record.read_id),read_record.signal,read_record.calibration.offset,read_record.calibration.scale])
            #0:read_id,1:signal,2:shift,3:scale
    return signals
def extract_move_from_bam(bam_path):
    seq_move=[]
    bamfile = pysam.AlignmentFile(bam_path, "rb",check_sq=False)
    try:
        for read in bamfile.fetch(until_eof=True):#暂时不使用索引，使用返回是空值
            #print(read.query_name)
            tags=dict(read.tags)
            mv_tag=tags['mv']
            ts_tag=tags['ts']
            sm_tag=tags["sm"]
            sd_tag=tags["sd"]
            #read.update({read.query_name:{"sequence":read.query_sequence,"stride":mv_tag[0],"mv_table":np.array(mv_tag[1:]),"num_trimmed":ts_tag,"shift":sm_tag,"scale":sd_tag}})
            seq_move.append([read.query_name,read.query_sequence,mv_tag[0],np.array(mv_tag[1:]),ts_tag,sm_tag,sd_tag])
    except ValueError:
        print('bam don\'t has index')
        for read in bamfile.fetch(until_eof=True,multiple_iterators=False):
            tags=dict(read.tags)
            mv_tag=tags['mv']
            ts_tag=tags['ts']
            sm_tag=tags["sm"]
            sd_tag=tags["sd"]
            seq_move.append([read.query_name,read.query_sequence,mv_tag[0],np.array(mv_tag[1:]),ts_tag,sm_tag,sd_tag])
            #0:read_id,1:sequence,2:stride,3:mv_table,4:num_trimmed,5:to_norm_shift,6:to_norm_scale
            #read[read.query_name] = {"sequence":read.query_sequence,"stride":mv_tag[0],"mv_table":np.array(mv_tag[1:]),"num_trimmed":ts_tag,"shift":sm_tag,"scale":sd_tag}
    return seq_move
def read_from_pod5_bam(pod5_path,bam_path,read_id=None):
    read=[]
    signal = extract_signal_from_pod5(pod5_path)
    seq_move = extract_move_from_bam(bam_path)
    if read_id is not None:
        for i in range(len(seq_move)):
            if seq_move[i][0]==read_id:
                if seq_move[i][1] is not None:
                    for j in range(len(signal)):
                        if signal[j][0]==seq_move[i][0]:
                            read.append([signal[j][0],signal[j][1],signal[j][2],signal[j][3],
                            seq_move[i][1],seq_move[i][2],seq_move[i][3],seq_move[i][4],seq_move[i][5],seq_move[i][6]])
        
    else:
        for i in range(len(seq_move)):
            if seq_move[i][1] is not None:
                for j in range(len(signal)):
                    if signal[j][0]==seq_move[i][0]:
                        read.append([signal[j][0],signal[j][1],signal[j][2],signal[j][3],
                        seq_move[i][1],seq_move[i][2],seq_move[i][3],seq_move[i][4],seq_move[i][5],seq_move[i][6]])
                #0:read_id,1:signal,2:to_pA_shift,3:to_pA_scale,4:sequence,5:stride,6:mv_table,7:num_trimmed,8:to_norm_shift,9:to_norm_scale
                    
                
    return read

In [20]:
bam_path = '/homeb/xiaoyf/data/HG002/example/bam/sorted_has_moves.bam'
bamfile = pysam.AlignmentFile(bam_path, "rb",check_sq=False)
bamfile.has_index()

True

In [14]:
for read in bamfile.get_index_statistics():
    #print(read)
    print(read)

In [18]:
for read in bamfile:
    print(read.query_name)

In [ ]:
seq_and_move_table=extract_move_from_bam(bam_path)

In [6]:
base2code_dna = {'A': 0, 'C': 1, 'G': 2, 'T': 3, 'N': 4,
                 'W': 4, 'S': 4, 'M': 4, 'K': 4, 'R': 4,
                 'Y': 4, 'B': 4, 'V': 4, 'D': 4, 'H': 4,
                 'Z': 4}  # set 4 for all bases except ACGT, for now
def norm_signal_read_id_dict(signal):
    shift_scale_norm={}
    signal_norm={}
    shift_scale_norm={}
    shift_scale_norm['shift']=(signal['to_norm_shift']/signal['to_pA_scale'])-signal['to_pA_shift']
    shift_scale_norm['scale']=(signal['to_norm_scale']/signal['to_pA_scale'])
    num_trimmed=signal["num_trimmed"]
    signal_norm=(signal['signal'][num_trimmed:] - shift_scale_norm['shift']) / shift_scale_norm['scale']        
    return signal_norm

def read_from_pod5_bam_dict(pod5_path,bam_path,read_id=None):
    read={}
    signal = extract_signal_from_pod5(pod5_path)
    seq_move = extract_move_from_bam(bam_path)
    if read_id is not None:
        if seq_move[read_id]['sequence'] is not None:
            if signal[read_id] is not None:
                read[read_id]={"sequence":seq_move[read_id]['sequence'],"signal":signal[read_id]['signal'],'mv_table':seq_move[read_id]['mv_table'],
                               "num_trimmed":seq_move[read_id]['num_trimmed'],"to_norm_shift":seq_move[read_id]['shift'],
                               "to_norm_scale":seq_move[read_id]['scale'],"stride":seq_move[read_id]['stride'],
                               'to_pA_shift':signal[read_id]['shift'],
                               'to_pA_scale':signal[read_id]["scale"]}
    else:
        for read_id in seq_move.keys():
            if seq_move[read_id]['sequence'] is not None:
                if signal[read_id] is not None:
                    read[read_id]={"sequence":seq_move[read_id]['sequence'],"signal":signal[read_id]['signal'],'mv_table':seq_move[read_id]['mv_table'],
                               "num_trimmed":seq_move[read_id]['num_trimmed'],"to_norm_shift":seq_move[read_id]['shift'],
                               "to_norm_scale":seq_move[read_id]['scale'],"stride":seq_move[read_id]['stride'],
                               'to_pA_shift':signal[read_id]['shift'],
                               'to_pA_scale':signal[read_id]["scale"]}
    return read



In [16]:
pod5_path = '/homeb/xiaoyf/data/HG002/example/pod5/output.pod5'
bam_path = '/homeb/xiaoyf/data/HG002/example/bam/has_moves.bam'
read=read_from_pod5_bam(pod5_path,bam_path)

[E::idx_find_and_load] Could not retrieve index file for '/homeb/xiaoyf/data/HG002/example/bam/has_moves.bam'


In [12]:
read[0]

['3239b4d9-0a7e-471c-86fe-e156b9f279a0',
 array([1284, 1066, 1050, ..., 1008, 1023,  920], dtype=int16),
 -243.0,
 0.1462070643901825,
 'ATGTATATGTAACCTACTTGGTTCAGTTACGTACTGCTGAATAAGTCTCACAATATTGATGGCTTTAAAGAGAGAAGTCCCCGGCCGGGCTGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCAAGGCAGGAGGATCACCTGAGGTCAAGAGTTCGAGACCAGCCTGGCCAACATGGTGAAACCTCCTCTCTACTAAAAATACAAAAATTAGCCAGGCATGGTGGCAGGTGCCTGTAATCCCAGCTACTTGGGAGGCTGAGGCAGGAGAATTGCTTGAACCTGGGAGGCAGAGATTGCAGTGAGCTGAGATCCCGCCACTGCAGTCCAGCCTGGGGGACAAGCAATACGTT',
 5,
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
        1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,

In [38]:
#feature=caculate_feature_one_one_for_each_base(read,21)
#feature['0000b1ad-fdaf-49e6-bc11-cbe93270e3a3'][0].keys()

dict_keys(['signal', 'std', 'mean', 'num'])

In [55]:
def _get_neighbord_feature_dict(feature,base_num):
    windows_size=base_num-1//2
    for read_id in feature.keys():
        for i in range(len(feature[read_id])):
            nbase=[]
            nstd=[]
            nmean=[]
            nsig=[]
            if i<windows_size:                   
                if i!=0:
                    for k in range(i):
                        nbase=nbase+list(feature[read_id][k]['base'])*feature[read_id][k]["num"]
                        nstd=nstd+list(feature[read_id][k]['std'])*feature[read_id][k]['num']
                        nmean=nmean+list(feature[read_id][k]['mean'])*feature[read_id][k]['num']
                        nsig=nsig+feature[read_id][k]['signal']
                nbase=nbase+list(feature[read_id][i]['base'])*(windows_size-i)*feature[read_id][i]['num']
                nbase=nbase+list(feature[read_id][i]['base'])*feature[read_id][i]['num']
                nstd=nstd+list(feature[read_id][i]['std'])*(windows_size-i)*feature[read_id][i]['num']
                nstd=nstd+list(feature[read_id][i]['std'])*feature[read_id][i]['num']
                nmean=nmean+list(feature[read_id][i]['mean'])*(windows_size-i)*feature[read_id][i]['num']
                nmean=nmean+list(feature[read_id][i]['mean'])*feature[read_id][i]['num']
                for k in range(i,i+windows_size):
                    nbase=nbase+list(feature[read_id][k]['base'])*feature[read_id][k]['num']
                    nstd=nbase+list(feature[read_id][k]['std'])*feature[read_id][k]['num']
                    nmean=nbase+list(feature[read_id][k]['mean'])*feature[read_id][k]['num']
                nsig=nsig+feature[read_id][i]['signal']*(windows_size-i)
                nsig=nsig+feature[read_id][i]['signal']
                for k in range(i,i+windows_size):
                    nsig=nsig+feature[read_id][k]['signal']
            elif (len(feature[read_id])-1)-i<windows_size:
                for k in range(i-windows_size,i):
                    nbase=nbase+list(feature[read_id][k]['base'])*feature[read_id][k]['num']
                    nstd=nstd+list(feature[read_id][k]['std'])*feature[read_id][k]['num']
                    nmean=nmean+list(feature[read_id][k]['mean'])*feature[read_id][k]['num']
                nbase=nbase+list(feature[read_id][i]['base'])*feature[read_id][i]['num']
                nstd=nstd+list(feature[read_id][i]['std'])*feature[read_id][i]['num']
                nmean=nmean+list(feature[read_id][i]['mean'])*feature[read_id][i]['num']
                for k in range(i-windows_size,i):
                    nsig=nsig+feature[read_id][k]['signal']
                nsig=nsig+feature[read_id][i]['signal']                   
                if i!=len(feature[read_id])-1:
                    for k in range(i,len(feature[read_id])-1):
                        nbase=nbase+list(feature[read_id][k]['base'])*feature[read_id][k]['num']
                        nstd=nstd+list(feature[read_id][k]['std'])*feature[read_id][k]['num']
                        nmean=nmean+list(feature[read_id][k]['mean'])*feature[read_id][k]['num']
                        nsig=nsig+feature[read_id][k]['signal']
                nbase=nbase+list(feature[read_id][i]['base'])*(windows_size-((len(feature[read_id])-1)-i))*feature[read_id][i]['num']
                nstd=nstd+list(feature[read_id][i]['std'])*(windows_size-((len(feature[read_id])-1)-i))*feature[read_id][i]['num']
                nmean=nmean+list(feature[read_id][i]['mean'])*(windows_size-((len(feature[read_id])-1)-i))*feature[read_id][i]['num']
                nsig=nsig+feature[read_id][i]['signal']*(windows_size-((len(feature[read_id])-1)-i))
            else:
                for k in range(i-windows_size,i):
                    nbase=nbase+list(feature[read_id][k]['base'])*feature[read_id][k]['num']
                    nstd=nstd+list(feature[read_id][k]['std'])*feature[read_id][k]['num']
                    nmean=nmean+list(feature[read_id][k]['mean'])*feature[read_id][k]['num']
                    nsig=nsig+feature[read_id][k]['signal']
                nbase=nbase+list(feature[read_id][i]['base'])*feature[read_id][i]['num']
                nstd=nstd+list(feature[read_id][i]['std'])*feature[read_id][i]['num']
                nmean=nmean+list(feature[read_id][i]['mean'])*feature[read_id][i]['num']
                nsig=nsig+feature[read_id][i]['signal']
                for k in range(i,i+windows_size):
                    nbase=nbase+list(feature[read_id][k]['base'])*feature[read_id][k]['num']
                    nstd=nstd+list(feature[read_id][k]['std'])*feature[read_id][k]['num']
                    nmean=nmean+list(feature[read_id][k]['mean'])*feature[read_id][k]['num']
                    nsig=nsig+feature[read_id][k]['signal']
            feature[read_id][i].update({'nbase':nbase,'nsig':nsig,'nstd':nstd,'nmean':nmean})

def caculate_batch_feature_for_each_base_dict(read_q,feature_q,base_num=0):
    feature={}
    print("extrac_features process-{} starts".format(os.getpid()))
    read_num = 0
    while True:
        if read_q.empty():
            time.sleep(10)
        read_batch=read_q.get()
        if read_batch == "kill":
            read_q.put("kill")
            break
        read_num += len(read_batch)
        for read_id in read_batch.keys():
            feature[read_id]=[]
            sequence = read_batch[read_id]['sequence']
            movetable = read_batch[read_id]['mv_table']
            stride = read_batch[read_id]['stride']
            #num_trimmed = read[read_id]['num_trimmed']
            trimed_signals = norm_signal_read_id(read_batch[read_id])#筛掉背景信号,norm
            move_pos = np.append(np.argwhere(movetable == 1).flatten(), len(movetable))
            #print(len(move_pos))
            for move_idx in range(len(move_pos) - 1):
                start, end = move_pos[move_idx], move_pos[move_idx + 1]
                signal=trimed_signals[(start * stride):(end * stride)].tolist()
                mean=np.mean(signal)
                std=np.std(signal)
                num=end-start
                #print(move_idx)        
                feature[read_id].append({'signal':signal,'std':str(std),'mean':str(mean),'num':int(num*stride),'base':sequence[move_idx]})
        #feature_q.append(feature)
        if base_num!=0:
            _get_neighbord_feature(feature,base_num)
        feature_q.put(feature)
        print("extrac_features process-{} ending, proceed {} read batch".format(os.getpid(), read_num))
            
        

def _prepare_read_dict(read_q,read,batch_size=1000):
    i=0
    read_batch={}
    for read_id in read.keys():
        #print(read_id)
        read_batch[read_id]={}
        read_batch[read_id].update(read[read_id])
        i=i+1
        if i==batch_size:
            i=0
            read_q.put(read_batch)
            read_batch={}
    read_q.put(read_batch)

def write_feature_dict(feature_q):
    print("write_process-{} starts".format(os.getpid()))
    with open('/homeb/xiaoyf/data/HG002/example/feature.txt','w') as f:
        while True:
            if feature_q.empty():
                time.sleep(10)
                continue
            feature = feature_q.get()
            if feature == "kill":
                print('write_process-{} finished'.format(os.getpid()))
                break
            for read_id in feature.keys():
                #f.write(read_id+'\t')
                for i in range(len(feature[read_id])):
                    f.write(str(feature[read_id][i]['nbase'])+'\t'+str(feature[read_id][i]['nsig'])+'\t'+str(feature[read_id][i]['nstd'])+
                            '\t'+str(feature[read_id][i]['nmean'])+'\n')
            f.flush()
            
        

def extract_feature():
    start = time.time()
    feature_q = Queue()
    read_q=Queue()
    _prepare_read(read_q,read,batch_size=500)
    feature_procs=[]
    nproc = 4
    read_q.put("kill")
    for _ in range(nproc):
        p = mp.Process(target=caculate_batch_feature_for_each_base, args=(read_q,feature_q,21,))
        p.daemon = True
        p.start()
        feature_procs.append(p)

    p_w = mp.Process(target=write_feature, args=(feature_q,))
    p_w.daemon = True
    p_w.start()

    for p in feature_procs:
        p.join()

    feature_q.put("kill")
    p_w.join()
    print("[main]extract_features costs %.1f seconds.." %(time.time() - start))

In [ ]:
read_q=Queue()
_prepare_read(read_q,read,batch_size=1000)
read_batch=read_q.get()
for read_id in read_batch:
    print(read_id)

In [62]:
#0:read_id,1:signal,2:std,3:mean,4:num,5:base 
def _get_neighbord_feature(feature,base_num):
    nfeature=[]
    windows_size=base_num-1//2
    for i in range(len(feature)):
        nbase=[]
        nstd=[]
        nmean=[]
        nsig=[]
        if i<windows_size:                   
            if i!=0:
                for k in range(i):
                    nbase=nbase+list(feature[k][5])*feature[k][4]
                    nstd=nstd+list(feature[k][2])*feature[k][4]
                    nmean=nmean+list(feature[k][3])*feature[k][4]
                    nsig=nsig+feature[k][1]
            nbase=nbase+list(feature[i][5])*(windows_size-i)*feature[i][4]
            nbase=nbase+list(feature[i][5])*feature[i][4]
            nstd=nstd+list(feature[i][2])*(windows_size-i)*feature[i][4]
            nstd=nstd+list(feature[i][2])*feature[i][4]
            nmean=nmean+list(feature[i][3])*(windows_size-i)*feature[i][4]
            nmean=nmean+list(feature[i][3])*feature[i][4]
            nsig=nsig+feature[i][1]*(windows_size-i)
            nsig=nsig+feature[i][1]
            for k in range(i,i+windows_size):
                nbase=nbase+list(feature[k][5])*feature[k][4]
                nstd=nbase+list(feature[k][2])*feature[k][4]
                nmean=nbase+list(feature[k][3])*feature[k][4]
                nsig=nsig+feature[k][1]
        elif (len(feature[i])-1)-i<windows_size:
            for k in range(i-windows_size,i):
                nbase=nbase+list(feature[k][5])*feature[k][4]
                nstd=nstd+list(feature[k][2])*feature[k][4]
                nmean=nmean+list(feature[k][3])*feature[k][4]
                nsig=nsig+feature[k][1]
            nbase=nbase+list(feature[i][5])*feature[i][4]
            nstd=nstd+list(feature[i][2])*feature[i][4]
            nmean=nmean+list(feature[i][3])*feature[i][4]        
            nsig=nsig+feature[i][1]                   
            if i!=len(feature[i])-1:
                for k in range(i,len(feature[i])-1):
                    nbase=nbase+list(feature[k][5])*feature[k][4]
                    nstd=nstd+list(feature[k][2])*feature[k][4]
                    nmean=nmean+list(feature[k][3])*feature[k][4]
                    nsig=nsig+feature[k][1]
            nbase=nbase+list(feature[i][5])*(windows_size-((len(feature[i])-1)-i))*feature[i][4]
            nstd=nstd+list(feature[i][2])*(windows_size-((len(feature[i])-1)-i))*feature[i][4]
            nmean=nmean+list(feature[i][3])*(windows_size-((len(feature[i])-1)-i))*feature[i][4]
            nsig=nsig+feature[i][1]*(windows_size-((len(feature[i])-1)-i))
        else:
            for k in range(i-windows_size,i):
                nbase=nbase+list(feature[k][5])*feature[k][4]
                nstd=nstd+list(feature[k][2])*feature[k][4]
                nmean=nmean+list(feature[k][3])*feature[k][4]
                nsig=nsig+feature[k][1]
            nbase=nbase+list(feature[i][5])*feature[i][4]
            nstd=nstd+list(feature[i][2])*feature[i][4]
            nmean=nmean+list(feature[i][3])*feature[i][4]
            nsig=nsig+feature[i][1]
            for k in range(i,i+windows_size):
                nbase=nbase+list(feature[k][5])*feature[k][4]
                nstd=nstd+list(feature[k][2])*feature[k][4]
                nmean=nmean+list(feature[k][3])*feature[k][4]
                nsig=nsig+feature[k][1]
        #feature[read_id][i].update({'nbase':nbase,'nsig':nsig,'nstd':nstd,'nmean':nmean})
        nfeature.append([feature[0],nbase,nsig,nstd,nmean])
        #0:read_id,1:nbase,2:nsig,3:nstd,4:nmean
    return nfeature
        
#0:read_id,1:signal,2:to_pA_shift,3:to_pA_scale,4:sequence,5:stride,6:mv_table,7:num_trimmed,8:to_norm_shift,9:to_norm_scale
def norm_signal_read_id(signal):
    shift_scale_norm=[]
    signal_norm=[]
    shift_scale_norm=[(signal[8]/signal[3])-signal[2],(signal[9]/signal[3])]
    #0:shift,1:scale
    #print('type:'.format(type(signal[1])))
    #print('是不是list:'.format(isinstance(signal[1],list)))
    #print('是不是tuple:'.format(isinstance(signal[1],tuple)))
    num_trimmed=signal[7]
    #print('num_trimmed:{} and signal:{}'.format(num_trimmed,signal[1]))
    #print('shift:{} and scale:{}'.format(shift_scale_norm[0],shift_scale_norm[1]))
    signal_norm=(signal[1][num_trimmed:] - shift_scale_norm[0]) / shift_scale_norm[1]        
    return signal_norm

def caculate_batch_feature_for_each_base(read_q,feature_q,base_num=0):
    print("extrac_features process-{} starts".format(os.getpid()))
    read_num = 0
    while True:
        if read_q.empty():
            time.sleep(10)
        read_batch=read_q.get()
        if read_batch == "kill":
            read_q.put("kill")
            break
        read_num += len(read_batch)
        feature=[]
        for read_one in tqdm(read_batch):
            #print(read_one)            
            sequence = read_one[4]
            stride = read_one[5]
            movetable = read_one[6]           
            #num_trimmed = read[read_id]['num_trimmed']
            trimed_signals = norm_signal_read_id(read_one)#筛掉背景信号,norm
            move_pos = np.append(np.argwhere(movetable == 1).flatten(), len(movetable))
            #print(len(move_pos))
            for move_idx in range(len(move_pos) - 1):
                start, end = move_pos[move_idx], move_pos[move_idx + 1]
                signal=trimed_signals[(start * stride):(end * stride)].tolist()
                mean=np.mean(signal)
                std=np.std(signal)
                num=end-start
                #print(move_idx)
                feature.append([read_one[0],signal,str(std),str(mean),int(num*stride),sequence[move_idx]])
                #0:read_id,1:signal,2:std,3:mean,4:num,5:base        
                #feature[read_id].append({'signal':signal,'std':str(std),'mean':str(mean),'num':int(num*stride),'base':sequence[move_idx]})
        #feature_q.append(feature)
        if base_num!=0:
            nfeature=_get_neighbord_feature(feature,base_num)
        feature_q.put(nfeature)
        print("extrac_features process-{} ending, proceed {} read batch".format(os.getpid(), read_num))
            
        

def _prepare_read(read_q,read,batch_size=1000):
    i=0
    read_batch=[]
    for read_one in read:
        read_batch.append(read_one)
        i=i+1
        if i==batch_size:
            i=0
            read_q.put(read_batch)
            read_batch=[]
    read_q.put(read_batch)

def write_feature(feature_q):
    print("write_process-{} starts".format(os.getpid()))
    with open('/homeb/xiaoyf/data/HG002/example/feature.txt','w') as f:
        while True:
            if feature_q.empty():
                time.sleep(10)
                continue
            features = feature_q.get()
            if features == "kill":
                print('write_process-{} finished'.format(os.getpid()))
                break
            for feature in features:
                #0:read_id,1:nbase,2:nsig,3:nstd,4:nmean
                #f.write(read_id+'\t')
                f.write(str(feature[1])+'\t'+str(feature[2])+'\t'+str(feature[3])+
                            '\t'+str(feature[4])+'\n')
                    
            f.flush()
            
        

def extract_feature(nproc = 4):
    start = time.time()
    feature_q = Queue()
    read_q=Queue()
    _prepare_read(read_q,read,batch_size=500)
    feature_procs=[]  
    read_q.put("kill")
    for _ in range(nproc):
        p = mp.Process(target=caculate_batch_feature_for_each_base, args=(read_q,feature_q,21,))
        p.daemon = True
        p.start()
        feature_procs.append(p)

    p_w = mp.Process(target=write_feature, args=(feature_q,))
    p_w.daemon = True
    p_w.start()

    for p in feature_procs:
        p.join()

    feature_q.put("kill")
    p_w.join()
    print("[main]extract_features costs %.1f seconds.." %(time.time() - start))

###logging,tqdm,yeld,detopt

In [25]:
[1,2,3,4,5][2:]

[3, 4, 5]

In [63]:
extract_feature(4)

extrac_features process-57171 starts
extrac_features process-57174 starts
extrac_features process-57179 startsextrac_features process-57184 starts

write_process-57190 starts


100%|██████████| 500/500 [09:42<00:00,  1.17s/it]
Process Process-49:
Traceback (most recent call last):
  File "/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
  File "/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_136090/527151495.py", line 144, in write_feature
    if feature == "kill":
UnboundLocalError: local variable 'feature' referenced before assignment


[main]extract_features costs 1179.7 seconds..


Traceback (most recent call last):
  File "/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/multiprocessing/connection.py", line 405, in _send_bytes
    self._send(buf)
  File "/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/xiaoyf/anac